In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM
from cfmining.criteria import PercentileCalculator, PercentileCriterion, NonDomCriterion
from cfmining.predictors import GeneralClassifier_Shap
from cfmining.action_set import ActionSet

%load_ext autoreload
%autoreload 2

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [4]:
VAL_RATIO = 1/7
TEST_RATIO = 0.3
SEED = 0

df = pd.read_csv("../data/german.csv")
X = df.drop("GoodCustomer", axis=1)
Y = df["GoodCustomer"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_RATIO, random_state=SEED, shuffle=True)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=VAL_RATIO, random_state=SEED, shuffle=True)

model = joblib.load("../models/german/LGBMClassifier.pkl")
isolation_tree = joblib.load("../models/german/IsolationForest.pkl")

In [5]:
action_set = ActionSet(X = X_train)
for feat in action_set:
    if feat.name in ['Age', 'JobClassIsSkilled', 'OwnsHouse', 'isMale', 'JobClassIsSkilled']:
        feat.mutable = False
    if feat.name in ['Single', 'ForeignWorker', 'RentsHouse']:
        feat.mutable = False
    if feat.name in ['LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'NumberOfOtherLoansAtBank', 'MissedPayments', 'CriticalAccountOrLoansElsewhere', 'OtherLoansAtBank', 'Unemployed', 'YearsAtCurrentJob_lt_1']:
        feat.mutable = True
    if feat.name in ['YearsAtCurrentHome', 'NumberOfLiableIndividuals', 'HasTelephone', 'CheckingAccountBalance_geq_0','CheckingAccountBalance_geq_200', 'SavingsAccountBalance_geq_100', 'SavingsAccountBalance_geq_500', 'NoCurrentLoan', 'HasCoapplicant', 'HasGuarantor', 'YearsAtCurrentJob_geq_4']:
        feat.mutable = True
    
    if feat.name == "LoanDuration":
        feat.step_type = "absolute"
        feat.step_size = 6
    
    #if feat.name == "LoanAmount":
    #    feat.step_size = 0.1
        
    if feat.name == "LoanAmount":
        feat.mutable = False

    feat.step_direction = 0
    feat.update_grid()

In [6]:
percCalc = PercentileCalculator(action_set = action_set)

In [51]:
model_shap = GeneralClassifier_Shap(model, X_train, Y_train, tree = True)

entrou aqui


In [8]:
denied_individ = model.predict(X_test) == 0
individual = X_test.iloc[denied_individ]

In [9]:
direc = [0, 0, 1, -1, -1, -1, 1, -1, 0, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 0]
criteria = NonDomCriterion(direc)

In [19]:
def get_solutions_mapocam2(individual):
    #percCalc = PercentileCalculator(action_set=action_set)
    #criteria = PercentileCriterion(individual, percCalc)
    mapocam = MAPOCAM2(action_set, individual.values, model_shap, max_changes=3, compare = criteria, clean_suboptimal=False)#, recursive=True)
    mapocam.fit()
    return mapocam.solutions

In [59]:
#percCalc = PercentileCalculator(action_set=action_set)
#criteria = PercentileCriterion(individual.iloc[0], percCalc)
mapocam = MAPOFCEM(action_set, individual.iloc[0].values, model_shap, max_changes=3, compare = criteria) #, recursive=True)
mapocam.fit()

In [60]:
mapocam.solutions

[array([   0,    1,   30,   47, 3959,    4,    3,    1,    1,    1,    0,
           0,    0,    0,    1,    0,    1,    0,    0,    0,    1,    0,
           1,    0,    0,    1,    1])]

In [54]:
model.predict_proba(mapocam.solutions[0].reshape(1, -1))

array([[0.45981401, 0.54018599]])

In [49]:
mapocam.outlier_detection_counter

0